In [ ]:
import torch
import numpy as np
import cv2
import os 
import pandas as pd
import albumentations as A
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from albumentations.pytorch.functional import img_to_tensor
from tensorboardX import SummaryWriter
from tqdm.notebook import tqdm

In [2]:
from helpers.config import load_config
from helpers.utils import create_optimizer, AverageMeter
from helpers.losses import miou_round, val_miou_round
from helpers import losses
conf = load_config('helpers/eff_conf.json') 

In [3]:
PTH = '../../oil_spill_dataset/'

In [4]:
train = os.path.join(PTH, 'train/images')
train_mask = os.path.join(PTH, 'train/labels')

valid = os.path.join(PTH, 'test/images')
valid_mask = os.path.join(PTH, 'test/labels')

# Create transforms

In [5]:
def create_train_transforms():
    #(650,1250) > 512x512 > 512x512 > (650x1250) > 
    
    return A.Compose([
        A.RandomCrop(640, 640, p=1.0),
        #A.ShiftScaleRotate(),
        A.HorizontalFlip()
    ])

def create_val_transforms():
    return A.Compose([
#        A.Crop(0,0,640,640, p=1.0)
    ])

# Convert RGB mask to 1D mask

In [6]:
def rgb_to_label(rgb_mask):
    color_to_label = {
        (0,0,0): 0,
        (0,255,255): 1,
        (255,0,0): 2,
        (153, 76,0): 3,
        (0,153,0): 4
    }
    height, width, _ = rgb_mask.shape
    label_mask = np.zeros((height, width), dtype = np.uint8)
    
    for color, label in color_to_label.items():
        indices = np.where(np.all(rgb_mask==color, axis=-1))
        #print(indices)
        label_mask[indices] = label
    return label_mask

In [7]:
rgb_mask = cv2.imread('../../oil_spill_dataset/train/labels/34_904,35_492_3.png')

In [8]:
label_mask = rgb_to_label(rgb_mask)

In [9]:
np.unique(label_mask)

array([0, 4], dtype=uint8)

# Create CSV with common file names

In [10]:
def common_ims(img_pth, mask_pth, mode):
    img_ids = os.listdir(img_pth)
    mask_ids = os.listdir(mask_pth)
    mask_ids = [s.replace('png', 'jpg') for s in mask_ids]
    
    img_df = pd.DataFrame({'filename': img_ids})
    mask_df = pd.DataFrame({'filename': mask_ids})

    merged_df = pd.merge(img_df, mask_df, on='filename')
    merged_df.to_csv(f'{mode}.csv', index=False)

In [11]:
common_ims(train, train_mask, 'train')
common_ims(valid, valid_mask, 'valid')

In [12]:
train_df = pd.read_csv('train.csv')[:50]
test_df = pd.read_csv('test.csv')

In [13]:
len(os.listdir(train)), len(os.listdir(train_mask))

(1014, 1022)

# Dataset

In [14]:
class OilDataset(Dataset):
    def __init__(self, df, mode='train', classes=None, augmentations = None, normalize = None):
        self.df = df['filename']
        self.mode = mode
        self.classes = classes
        self.augmentations = augmentations
        self.normalize = normalize
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image = cv2.imread(os.path.join(f'../../oil_spill_dataset/{self.mode}/images/', self.df[idx]))
        mask = cv2.imread(os.path.join(f'../../oil_spill_dataset/{self.mode}/labels/', self.df[idx].replace('jpg', 'png')))
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = rgb_to_label(mask)
        mask[mask != 1] = 0
        
        if self.augmentations:
            sample = self.augmentations(image=image, mask=mask)
            #image, mask = sample['image'], sample['mask']
        
        mask = np.zeros((self.classes, *sample["mask"].shape[:2]))
        for i in range(self.classes):
            mask[i, sample["mask"] == i] = 1
        
        sample['img_name'] = os.path.join(f'../../oil_spill_dataset/{self.mode}/images/', self.df[idx])
        sample['mask_orig'] = sample['mask']
        sample['mask'] = torch.from_numpy(np.ascontiguousarray(mask)).float()
        sample['image'] = img_to_tensor(np.ascontiguousarray(sample['image']), self.normalize)
        return sample
        
        

In [15]:
dataset = OilDataset(train_df, mode='train', classes = 2, augmentations=create_train_transforms(), normalize = conf['input']['normalize'])

In [16]:
dataset[0]['image'].shape, dataset[0]['img_name'], dataset[0]['mask'].shape, dataset[0]['mask_orig'].shape


(torch.Size([3, 640, 640]),
 '../../oil_spill_dataset/train/images/43_538,29_653_2.jpg',
 torch.Size([2, 640, 640]),
 (640, 640))

In [17]:
np.unique(dataset[0]['mask_orig'])

array([0, 1], dtype=uint8)

# Config

# Train

In [18]:
def train_epoch(current_epoch, loss_functions, model, optimizer, scheduler, train_data_loader, 
                summary_writer, conf):
    losses = AverageMeter()
    mious = AverageMeter()
    iterator = tqdm(train_data_loader)
    model.train()
    if conf["optimizer"]["schedule"]["mode"] == "epoch":
        scheduler.step(current_epoch)
    for i, sample in enumerate(iterator):
        imgs = sample["image"].cuda()
        masks = sample["mask"].cuda().float()
        masks_orig = sample["mask_orig"].cuda().float()
        out_mask = model(imgs)
        with torch.no_grad():
            pred = torch.softmax(out_mask, dim=1)
            argmax = torch.argmax(pred, dim=1)
            ious = miou_round(argmax, masks_orig).item()

        mious.update(ious, imgs.size(0))

        mask_loss = loss_functions["mask_loss"](out_mask, masks.contiguous())
        loss = mask_loss
        losses.update(loss.item(), imgs.size(0))
        iterator.set_description(
            "epoch: {}; lr {:.7f}; Loss ({loss.avg:.4f}); miou ({miou.avg:.4f}); ".format(
                current_epoch, scheduler.get_lr()[-1], loss=losses, miou=mious))
        optimizer.zero_grad()
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        #torch.cuda.synchronize()

        if conf["optimizer"]["schedule"]["mode"] in ("step", "poly"):
            scheduler.step(i + current_epoch * len(train_data_loader))
   
    for idx, param_group in enumerate(optimizer.param_groups):
        lr = param_group['lr']
        summary_writer.add_scalar('group{}/lr'.format(idx), float(lr), global_step=current_epoch)
    summary_writer.add_scalar('train/loss', float(losses.avg), global_step=current_epoch)

In [19]:

def segment_single_image(image, model, conf):
           
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = img_to_tensor(np.ascontiguousarray(image), conf['input']['normalize'])
        
        with torch.no_grad():
            imgs = torch.unsqueeze(image, 0).cuda().float()
            output = model(imgs)
            pred = torch.softmax(output, dim=1)
            mask = pred.squeeze(0).permute(1,2,0).cpu().numpy().astype(np.float)
        return mask

def cropper_validator(img, model, conf):
    
    # Define the patch size and overlap
    patch_size = (640, 640)
    overlap = 0.7
    #print(img.shape)
    # Load the semantic segmentation model
    segmentation_map = np.zeros((img.shape[0], img.shape[1], 2), dtype=np.float32)
    #print(segmentation_map.shape)
    # Loop over the image patches
    for y in range(0, img.shape[0], int(patch_size[0] * overlap)):
        for x in range(0, img.shape[1], int(patch_size[1] * overlap)):

            patch = img[y:y+patch_size[0], x:x+patch_size[1]]
            
            if patch.shape[1] < patch_size[1] and patch.shape[0] == patch_size[0]:
                patch = img[y:y+patch_size[0], img.shape[1]-patch_size[1]:img.shape[1]]
                patch_seg_map = segment_single_image(patch, model, conf)
                segmentation_map[y:y+patch_size[0], img.shape[1]-patch_size[1]:img.shape[1]] += patch_seg_map
                #print(segmentation_map.shape)
            elif patch.shape[0] < patch_size[0] and patch.shape[1] == patch_size[1]:
                patch = img[img.shape[0]-patch_size[0]:img.shape[0], x:x+patch_size[1]]
                patch_seg_map = segment_single_image(patch, model, conf)
                #patch_seg_map = segmenter.segment_single_image(patch, 'img')
                segmentation_map[img.shape[0]-patch_size[0]:img.shape[0], x:x+patch_size[1]] += patch_seg_map
            elif patch.shape[0] < patch_size[0] and patch.shape[1] < patch_size[1]:
                patch = img[img.shape[0]-patch_size[0]:img.shape[0], img.shape[1]-patch_size[1]:img.shape[1]]
                #patch_seg_map = segmenter.segment_single_image(patch, 'img')
                patch_seg_map = segment_single_image(patch, model, conf)
                segmentation_map[img.shape[0]-patch_size[0]:img.shape[0], img.shape[1]-patch_size[1]:img.shape[1]] += patch_seg_map
            else:
                patch_seg_map = segment_single_image(patch, model, conf)
                segmentation_map[y:y+patch_size[0], x:x+patch_size[1]] += patch_seg_map
    
    return segmentation_map #probability map 2x650x1250 (0,...)

def validate(net, predictions_dir, conf):
    os.makedirs(predictions_dir, exist_ok=True)
    preds_dir = predictions_dir + "/predictions"
    os.makedirs(preds_dir, exist_ok=True)
    mious = []
    oil_class_iou = []
    with torch.no_grad():
        for filename in test_df.filename:
            
            image = cv2.imread(os.path.join(f'../../oil_spill_dataset/test/images/', filename))
            mask = cv2.imread(os.path.join(f'../../oil_spill_dataset/test/labels/', filename.replace('jpg', 'png')))
            mask = rgb_to_label(mask)
            mask[mask != 1] = 0
            
            mask = torch.tensor(mask) 
            
            map_preds = cropper_validator(image, net, conf)
            output = torch.tensor(map_preds).permute(2,0,1)
            #print(map_preds.shape)

            argmax = torch.argmax(output, dim=0) # 650x1250 #0,1,1,,0,1,1, 0
            
            #print(argmax.shape, mask.shape)
            for i in range(output.shape[0]):
                d, ious = val_miou_round(argmax, mask)
                mious.append(d.item())
                oil_class_iou.append(ious[1].item())

    print(np.mean(oil_class_iou))
    return np.mean(mious)
def evaluate_val(output_dir, data_val, miou_best, model, snapshot_name, current_epoch, optimizer, summary_writer,
                 predictions_dir, conf):
    print("Test phase")
    model = model.eval()
        
    miou = validate(model, predictions_dir, conf)
    summary_writer.add_scalar('val/miou', float(miou), global_step=current_epoch)
    if miou > miou_best:
        if output_dir is not None:
            torch.save({
                'epoch': current_epoch + 1,
                'state_dict': model.state_dict(),
                'miou_best': miou,

            }, output_dir + '/'+snapshot_name + "_best_miou.pt")
        miou_best = miou
    torch.save({
        'epoch': current_epoch + 1,
        'state_dict': model.state_dict(),
        'miou_best': miou_best,
    }, output_dir +'/' + snapshot_name + "_last.pt")
    print("miou: {}, miou_best: {}".format(miou, miou_best))
    return miou_best

In [20]:
import segmentation_models_pytorch as smp
model = smp.Unet(encoder_name='efficientnet-b4', classes=2) #, encoder_weights=None)
# model = smp.Unet(encoder_name='efficientnet-b4', classes=5, encoder_weights=None)
# checkpoint = torch.load('best_oil.pt', map_location='cpu')
# state_dict = checkpoint['state_dict']
# model.load_state_dict(state_dict, strict=False)
# model.segmentation_head[0] = torch.nn.Conv2d(16, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))



In [21]:
#model.segmentation_head[0] = torch.nn.Conv2d(16, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [22]:
#model

In [23]:
model = model.cuda()

In [24]:
mask_loss_function = losses.__dict__[conf["mask_loss"]["type"]](**conf["mask_loss"]["params"]).cuda()
loss_functions = {"mask_loss": mask_loss_function}
optimizer, scheduler = create_optimizer(conf['optimizer'], model)

miou_best = 0
start_epoch = 0
batch_size = conf['optimizer']['batch_size']
#print(train_lst)
data_train = OilDataset(train_df,
                        mode='train', 
                        classes = 2, 
                        augmentations=create_train_transforms(), 
                        normalize = conf['input']['normalize'])
data_val = OilDataset(test_df,
                        mode='test', 
                        classes = 2, 
                        augmentations=create_val_transforms(), 
                        normalize = conf['input']['normalize'])
train_sampler = None

In [25]:
train_data_loader = DataLoader(data_train, batch_size=batch_size, num_workers=8,
                                   shuffle=train_sampler is None, sampler=train_sampler, pin_memory=False,
                                   drop_last=True)
#print(data_train.__getitem__(1)['image'].shape)
#print(data_train.__getitem__(1)['mask'].shape)
val_batch_size = 1
val_data_loader = DataLoader(data_val, batch_size=val_batch_size, num_workers=8, shuffle=False,
                             pin_memory=False)

In [26]:
#!mkdir predictions

In [27]:
output_dir = 'outputs'
predictions_path = 'predictions'

In [28]:
logdir = 'logs'
summary_writer = SummaryWriter(logdir + '/' + 'segment_' + conf['encoder'])

In [29]:
start_epoch = 0
current_epoch=0

In [30]:
snapshot_name = "{}{}_{}".format('segment_', conf['network'], conf['encoder'])

In [ ]:
for epoch in range(start_epoch, conf['optimizer']['schedule']['epochs']):
        if train_sampler:
            train_sampler.set_epoch(epoch)

        model_encoder_stages = model.encoder
        model_encoder_stages.train()
        for p in model_encoder_stages.parameters():
            p.requires_grad = True
        train_epoch(current_epoch, loss_functions, model, 
                    optimizer, scheduler, train_data_loader, summary_writer, conf)

        model = model.eval()
        
        torch.save({
            'epoch': current_epoch + 1,
            'state_dict': model.state_dict(),
            'miou_best': miou_best,
        }, output_dir + '/' + snapshot_name + "_last")
        preds_dir = os.path.join(predictions_path, snapshot_name)
        miou_best =  evaluate_val(output_dir, val_data_loader, miou_best, model, snapshot_name,
                      current_epoch,
                      optimizer, summary_writer,
                      preds_dir, conf)
        current_epoch += 1


# 1d to RGB

In [ ]:
import numpy as np
import cv2

def convert_mask(mask):
   
    color_mappings = {
        0: (0, 0, 0),
        1: (0, 255, 255),
        2: (255, 0, 0),
        3: (153, 76, 0),
        4: (0, 153, 0)
    }

   
    height, width= mask.shape
    rgb_mask = np.zeros((height, width, 3), dtype=np.uint8)

   
    for class_idx, color in color_mappings.items():
        class_mask = (mask == class_idx)
        rgb_mask[class_mask] = color

    
    return rgb_mask


In [ ]:
img1 = cv2.imread('predictions/segment_resnet34_resnet34/predictions/test_42_308,18_108_1.jpg_mask_orig.png')
img2 = cv2.imread('predictions/segment_resnet34_resnet34/predictions/test_42_308,18_108_1.jpg_prediction.png')

In [ ]:
img1 = img1/64
img2 = img2/64

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(img2)

In [ ]:
plt.imshow(convert_mask(img1[:,:,1]))

In [ ]:
plt.imshow(convert_mask(img2[:,:,1]))

In [ ]:
img[:,:,1].shape

In [ ]:
# 650x1250 > 